In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

from collections import Counter,defaultdict
from itertools import chain, combinations

import warnings
warnings.filterwarnings('ignore')

import imshow
from wordcloud import WordCloud
from PIL import Image

from itertools import chain
from collections import Counter
import pickle

import re
import sent2vec
import hnswlib
from gensim.models import Word2Vec
from gensim import corpora


import math

from py_file.total_processing import konlpy_preprocessing
kp = konlpy_preprocessing()

In [2]:
# konlpy_total을 통해 데이터를 통해서 word2vec 모델을 생성 
train = pd.read_json('data/1108_new_train.json')
train.head(3)

,tags,id,new_tags,plylst_title,songs,like_cnt,updt_date
0,[락],61281,"[여행, 락]",여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,"[회상, 추억]",요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"[편하, 까페, 잔잔]","편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


# 1. Word2Vec 모델 생성

자연어 전처리를 통해 나온 new_tag를 이용해  word2vec의 모델을 생성함

In [3]:
# w2v 모델 생성
w2v_model = Word2Vec(sentences=train['new_tags'].tolist(),vector_size=100,window=5,min_count=0,workers=6,sg=0,
                     negative=16)

# w2v 모델 저장
w2v_model.save('model/w2v.model')

# 2. KNN

- knn을 통해 단어의 유사도를 판별함
- 기존의 knn은 Euclidean 거리를 이용해 유사도를 계산하지만 이 방법에서는 Cosine 거리를 이용해 유사도를 계산함
- 현재 데이터를 보면 플레이리스트 new_tags에 하나의 태그만 있을 수도 여러 개의 태그가 있을 수도 있다. 
    - 그렇기 때문에 플레이리스트의 유사도를 판별할 때 여러 개의 태그가 있는 경우 vector의 평균으로 유사도를 계산함

- 아래에서는 new_tags, tags, playlist_title을 이용해 총 3가지의 knn모델을 생성한다

## 2_1 KNN 모델 학습을 위한 전처리 

- 아래의 방법은 각각 new_tags, tags, playlist_title을 이용함
- 각각의 데이터를 이용해 사용가능한 플레이리스트를 뽑아 낸다
- 없는 단어가 있다면 vector의 값을 np.zeros(100)을 통해 생성함
    - word2vec 모델을 만들 때 vector_size=100을 사용했기 때문에 똑같이 np.zeros(100)을 생성함

In [11]:
# new_tags
new_tags_emb = []
for i in train['new_tags'].tolist():
    a = []
    try:
        for j in i :
            vec = w2v_model.wv.get_vector(j)
            a.append(vec)
        # x.append(a)
        vec_ave = (sum(a))/len(a)
        new_tags_emb.append(vec_ave)
    except ZeroDivisionError:
        new_tags_emb.append(np.zeros(100))



new_tags_idx_to_emb = {}
for idx, emb in enumerate(new_tags_emb):
    if emb.sum() != 0:
        new_tags_idx_to_emb[idx] = emb



# tags
tags_emb = []
for i in train['tags'].tolist():
    a = []
    try:
        for j in i :
            vec = w2v_model.wv.get_vector(j)
            a.append(vec)
        # x.append(a)
        vec_ave = (sum(a))/len(a)
        tags_emb.append(vec_ave)
    except ZeroDivisionError and KeyError:
        tags_emb.append(np.zeros(100))



tags_idx_to_emb = {}
for idx, emb in enumerate(tags_emb):
    if emb.sum() != 0:
        tags_idx_to_emb[idx] = emb



# title
titles_emb = []
for i in train['plylst_title'].tolist():
    a = []
    try:
        for j in i :
            vec = w2v_model.wv.get_vector(j)
            a.append(vec)
        # x.append(a)
        vec_ave = (sum(a))/len(a)
        titles_emb.append(vec_ave)
    except ZeroDivisionError and KeyError:
        titles_emb.append(np.zeros(100))


titles_idx_to_emb = {}
for idx, emb in enumerate(tags_emb):
    if emb.sum() != 0:
        titles_idx_to_emb[idx] = emb

## 2_2 knn 모델 생성

- Cosine 거리를 통해 유사도를 계산함
- word2vec에서 vector_size=100으로 해주었기 때문에 dim 또한 100으로 설정

In [ ]:
# new_tags
p_new_tags = hnswlib.Index(space = 'cosine', dim = 100)
p_new_tags.init_index(max_elements = len(new_tags_idx_to_emb), 
                   ef_construction = 100, M = 16, random_seed = 100)
p_new_tags.add_items(list(new_tags_idx_to_emb.values()), list(new_tags_idx_to_emb.keys()))


# tags
p_tags = hnswlib.Index(space = 'cosine', dim = 100)
p_tags.init_index(max_elements = len(tags_idx_to_emb), 
                   ef_construction = 100, M = 16, random_seed = 100)
p_tags.add_items(list(tags_idx_to_emb.values()), list(tags_idx_to_emb.keys()))


# title
p_titles = hnswlib.Index(space = 'cosine', dim = 100)
p_titles.init_index(max_elements = len(titles_idx_to_emb), 
                   ef_construction = 100, M = 16, random_seed = 100)
p_titles.add_items(list(titles_idx_to_emb.values()), list(titles_idx_to_emb.keys()))

## 2_3. 모델 저장

In [12]:
# 모델 저장
pickle.dump(p_tags, open('/home/ubuntu/workspace/jh/project/model/w2v_p_tags.pickle', 'wb') )
pickle.dump(p_titles, open('/home/ubuntu/workspace/jh/project/model/w2v_p_titles.pickle', 'wb') )
pickle.dump(p_new_tags, open('/home/ubuntu/workspace/jh/project/model/w2v_p_new_tags.pickle', 'wb') )